<a href="https://colab.research.google.com/github/Coding-bot007/machine-learning/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install fastapi uvicorn
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256380 sha256=d219839bef213accf4fe4a630d98de79e523eeccc5a2a1e53864cbeef9386f78
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=72486e29827df90291a5850e9310f07a453b4b448970b43f1c075ac9396f5cfe
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d43

In [7]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [8]:
app = FastAPI()


In [9]:
import pickle

# Load the pre-trained model
with open('semantic_similarity.sav', 'rb') as file:
    loaded_model = pickle.load(file)


In [13]:
!pip install transformers
from pydantic import BaseModel
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

class InputData(BaseModel):
    sentence1: str
    sentence2: str

app = FastAPI()

# Load the pre-trained BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

@app.post("/predict/")
async def predict_similarity(data: InputData):
    # Tokenize the input sentences
    tokens1 = tokenizer(data.sentence1, return_tensors="pt")
    tokens2 = tokenizer(data.sentence2, return_tensors="pt")

    # Get the sentence embeddings using the pre-trained BERT model
    with torch.no_grad():
        output1 = model(**tokens1)
        output2 = model(**tokens2)

    sentence_embedding1 = torch.mean(output1.last_hidden_state, dim=1)  # Average pooling
    sentence_embedding2 = torch.mean(output2.last_hidden_state, dim=1)

    # Calculate the cosine similarity between the sentence embeddings
    cosine_sim = cosine_similarity(sentence_embedding1.numpy(), sentence_embedding2.numpy())

    # Convert the similarity score to a float value
    similarity_score = float(cosine_sim[0][0])

    return {"sentence1": data.sentence1, "sentence2": data.sentence2, "similarity_score": similarity_score}



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.8 MB/s eta 0:00:00


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


Public URL: https://8d54-34-83-65-82.ngrok.io


INFO:     Started server process [256]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     34.125.39.98:0 - "POST / HTTP/1.1" 404 Not Found
INFO:     34.125.39.98:0 - "POST / HTTP/1.1" 404 Not Found
INFO:     34.125.39.98:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     34.125.39.98:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     34.125.39.98:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     34.125.39.98:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     34.125.39.98:0 - "POST /predict/ HTTP/1.1" 200 OK
